In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Word Counts").getOrCreate()

In [2]:
from pyspark.sql.functions import lit

In [3]:
maps_california = spark.read.json("review-California.json")

In [4]:
maps_florida = spark.read.json("review-Florida.json")

In [5]:
maps_newyork = spark.read.json("review-New_York.json")

In [6]:
maps_metadata = spark.read.json("metadatasitios.json")

In [6]:
maps_metadata.count()

3025011

Proceso de eliminar registros duplicados

In [7]:
maps_CA =maps_california.dropDuplicates()

In [8]:
maps_FL = maps_florida.dropDuplicates()

In [9]:
maps_NY = maps_newyork.dropDuplicates()

In [10]:
maps_metadata_un =maps_metadata.dropDuplicates()

Se agrega el estado a los dataframe de maps review estados NY, florida, California

In [11]:
def agregar_estado(df,columna,valor):
    return df.withColumn(columna,lit(valor))

In [12]:
ny_conestado = agregar_estado(maps_NY,'estado','NY')

In [13]:
california_conestado = agregar_estado(maps_CA,'estado','CA')

In [14]:
florida_conestado = agregar_estado(maps_FL,'estado','FL')

Se eliminan registros nulos de la columna text para cada estado

In [15]:
ny_conestadot = ny_conestado.filter(ny_conestado['text'].isNotNull())

In [16]:
california_conestadot = california_conestado.filter(california_conestado['text'].isNotNull())

In [17]:
florida_conestadot = florida_conestado.filter(florida_conestado['text'].isNotNull())

se concatenan por filas los dataframe de los estados california, florida, NY

In [18]:
from functools import reduce
dataframes= [ny_conestadot,california_conestadot,florida_conestadot]

def union_dataframes(ny_conestadot, california_conestadot): #funcion que concatena todos los df requeridos
    return ny_conestadot.union(california_conestadot)

# Utilizar reduce() para unir los DataFrames
review_estados = reduce(union_dataframes, dataframes)

# Mostrar el DataFrame resultante
#review_estados.show(2)

al dataframe de maps_metadata le agregamos la columna de "estado"

In [19]:
metadata_con_state = review_estados.select(["gmap_id","estado"]).join(maps_metadata_un,on="gmap_id",how="inner")

In [26]:
metadata_con_state.count()

7996025

In [20]:
metadata_unicos =metadata_con_state.dropDuplicates()
metadata_unicos.count()

203023

In [21]:
review_estados.count()

4598701

In [24]:
 metadata_unicos.coalesce(1).write.json("output/metadata_final.json")

se hace prueba con dataframe de metadata se extrae category para desanidar las listas

In [25]:
category_prueba = spark.read.csv('metadata_1json_maps.csv',sep=';',header=True,inferSchema=True)

In [27]:
category_prueba.show(2)

+---------------+--------------------+--------------------+-----------+----------+------------+--------------------+----------+--------------+-----+--------------------+--------------------+-----------------+--------------------+--------------------+
|           name|             address|             gmap_id|description|  latitude|   longitude|            category|avg_rating|num_of_reviews|price|               hours|                MISC|            state|    relative_results|                 url|
+---------------+--------------------+--------------------+-----------+----------+------------+--------------------+----------+--------------+-----+--------------------+--------------------+-----------------+--------------------+--------------------+
|Porter Pharmacy|Porter Pharmacy, ...|0x88f16e41928ff68...|       null|   32.3883|    -83.3571|        ['Pharmacy']|       4.9|            16| null|[['Friday', '8AM–...|{'Service options...|Open ⋅ Closes 6PM|['0x88f16e4192943...|https://www.googl.

In [34]:
solo_category= category_prueba.select('gmap_id','category')

In [35]:
solo_category.show(10)

+--------------------+--------------------+
|             gmap_id|            category|
+--------------------+--------------------+
|0x88f16e41928ff68...|        ['Pharmacy']|
|0x80c2c98c0e3c16f...|['Textile exporter']|
|0x80c2c778e3b73d3...|['Korean restaura...|
|0x80c2c89923b27a4...|    ['Fabric store']|
|0x80c2c632f933b07...|    ['Fabric store']|
|0x80c2cf163db6bc8...|    ['Fabric store']|
|0x80dd2b4c8555edb...|      ['Restaurant']|
|0x87ec238afdd8837...|['Nail salon', 'W...|
|0x87ec235c54d25b3...|['Bakery', 'Healt...|
|0x87ee85ac3cb23b4...|['Greeting card s...|
+--------------------+--------------------+
only showing top 10 rows



In [30]:
from pyspark.sql.functions import explode

In [38]:
from pyspark.sql.functions import posexplode

In [39]:
solo_category.select('gmap_id',posexplode('category').alias('indice','category'))

AnalysisException: cannot resolve 'posexplode(category)' due to data type mismatch: input to function explode should be array or map type, not string;
'Project [gmap_id#352, posexplode(category#356) AS Buffer(indice, category)]
+- Project [gmap_id#352, category#356]
   +- Relation [name#350,address#351,gmap_id#352,description#353,latitude#354,longitude#355,category#356,avg_rating#357,num_of_reviews#358,price#359,hours#360,MISC#361,state#362,relative_results#363,url#364] csv
